In [15]:
import psycopg2

pg_host = "localhost"
pg_database = "graph"
pg_user = "postgres"
pg_password = "postgres"

# Establish PostgreSQL connection
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

In [5]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain

load_dotenv()

True

In [7]:
template1 = """
You are expert software analyst
Sentence 1 : {sentence1}

Sentence 2: {sentence2}

From sentence 1 and sentence 2, is that both refers in same feature of software requirement
Answer must be (yes/no), dont give any other answer than yes or no

"""

prompt1 = ChatPromptTemplate.from_template(template1)

model = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv('OPEN_API_KEY'), temperature=0)

chain1 = prompt1 | model

chain_one = LLMChain(llm=model, prompt=prompt1, output_key="response")

overal_chain = SequentialChain(
    chains=[chain_one],
    input_variables=["sentence1", "sentence2"],
    output_variables=["response"]
)

In [16]:
# Execute query to select similarities
pg_cursor.execute("""
    select id1, ds_source.sentence as source, id2, ds_target.sentence as target from distinct_pair
    inner join datasets as ds_source on ds_source.id = id1
    inner join datasets as ds_target on ds_target.id  = id2 
    group by id1, ds_source.id, id2, ds_target.id
""")

# Fetch all results
distinct = pg_cursor.fetchall()

In [19]:
for data in distinct[101:]:
    message = overal_chain({"sentence1": data[1], "sentence2": data[3]})
    response = message['response']
    if response.strip().lower() == 'yes':
        pg_cursor.execute("INSERT INTO prompt_matching (source_id, target_id, model, is_match) VALUES (%s, %s, %s, %s)", (data[0], data[2], 'gpt-4o-mini', True))
    else:
        pg_cursor.execute("INSERT INTO prompt_matching (source_id, target_id, model, is_match) VALUES (%s, %s, %s, %s)", (data[0], data[2], 'gpt-4o-mini', False))
    pg_conn.commit()